<a href="https://colab.research.google.com/github/Sukhwinder9813/EmojiPredictionfromTweet/blob/master/EmojiPredictorfromTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
import nltk
nltk.download('popular')

In [ ]:
!pip install emoji
import emoji


In [ ]:
from keras.preprocessing.text import Tokenizer # https://keras.io/zh/preprocessing/text/

In [ ]:
from nltk.corpus import stopwords # https://www.nltk.org/book/ch02.html 页面搜索stopwords


In [ ]:
data=pd.read_csv('Train.csv')

In [ ]:
x_train=data['TEXT'].values

In [ ]:
y_train=data['Label'].values

In [ ]:
stopwords.words('english')

In [10]:
smoothtweets=[]
stopper=set(stopwords.words('english')) # set() 生成无序不重复集 # stopwords.words() 列出无实意词（停止词）
for tweets in x_train:
    words=tweets.split(" ") #以空格为分隔符 分割所有词
    str = ""
    for word in words:
        if word[0] != "@" and word not in stopper: # 第一个字符不是@且不是停止词
            if word[0] == "#":
                word = word[1:] # 去除词前#
        str += word + " " # 去除停止词、词前#和第一个字符是@的词 重新形成一句话
    smoothtweets.append(str) # 将经过处理的句子放入list

In [11]:
smoothtweets[2]

'Been friends since 7th grade. Look at us now we all following our dreams doing what we love and…\r\n '

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=" ", lower=True)
tokenizer.fit_on_texts(smoothtweets)
newsmoothtweets=[]
newsmoothtweets=tokenizer.texts_to_sequences(smoothtweets)

In [ ]:
print(newsmoothtweets[3])

In [ ]:
from keras.preprocessing import sequence
X_train=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [ ]:
from keras.layers import *
from keras.models import Sequential 

In [ ]:
from keras.utils import to_categorical

In [ ]:
Y_train=to_categorical(y_train)

In [ ]:
Y_train.shape

In [ ]:
X_train.shape

In [ ]:
wordembedd={}
f=open('glove.6B.50d.txt',"r",encoding='utf-8')
for line in f:
  words=line.split()
  word=words[0]
  coefs=np.asarray(words[1:],dtype='float')
  wordembedd[word]=coefs
f.close()

In [ ]:
def populate_weight_matrix(vocab, raw_embedding):
    # Create weight matrix from pre-trained embeddings
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, 50))
    for word, i in vocab.items():
        if word in raw_embedding:
            weight_matrix[i] = raw_embedding[word]
    return weight_matrix

In [ ]:
vocab=tokenizer.word_index

In [ ]:
len(vocab)
import math

In [ ]:
weight_matrix=populate_weight_matrix(vocab,wordembedd)

In [ ]:
weight_matrix[8]

In [ ]:
max_length = math.ceil(sum([len(s.split(" ")) for s in smoothtweets])/len(smoothtweets))
vocab_size=len(vocab)+1
model=Sequential()
model.add(Embedding(vocab_size,50,weights=[weight_matrix],input_length=max_length+2,trainable=True,))
model.add(LSTM(128,dropout=0.2,return_sequences=True))
model.add(LSTM(128,dropout=0.2))
model.add(Dense(20,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,Y_train,epochs=5,batch_size=128,shuffle=True,validation_split=0.15)

In [ ]:
model.evaluate(X_train,Y_train)

In [ ]:
model.predict_classes(X_train[1:13])

In [ ]:
smoothtweets[9]

In [ ]:
df=pd.read_csv('Mapping.csv')

In [ ]:
df.head(20)

In [ ]:
data['Label'][9]

In [ ]:
df3=pd.read_csv('Test.csv')

In [ ]:
df3.head()

In [ ]:
xtest=df3['TEXT']


In [ ]:
smoothTESTtweets=[]
stopper=set(stopwords.words('english'))
for tweets in xtest:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothTESTtweets.append(str)

In [ ]:
newsmoothtweets=tokenizer.texts_to_sequences(smoothTESTtweets)

In [ ]:
newsmoothtweets[2]

In [ ]:
x_test=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [ ]:
x_test[3]

In [ ]:
data=model.predict_classes(x_test)


In [ ]:
for i in range(20,35,1):
  print(df['emoticons'][data[i]])
  print(smoothTESTtweets[i])

In [ ]:
smoothTESTtweets[2]

In [ ]:
xtest = ["I love you"]

smoothTESTtweets=[]
stopper=set(stopwords.words('english'))
for tweets in xtest:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothTESTtweets.append(str)

newsmoothtweets=tokenizer.texts_to_sequences(smoothTESTtweets)

x_test=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')
data=model.predict_classes(x_test)

In [ ]:
print(df['emoticons'][data[0]])
print(smoothTESTtweets[0])